## Valores que faltan (*missing values*)

  * Los valores que faltan pueden estar codificados de muchas maneras: "-", "?", "-9999", "N/A", "NA", etc...
  
  * numpy tiene el valor especial `np.NaN`, pandas `pd.NA`
  
  * En general es parte del preproceso identificar qué representa un valor que falta y reemplazarlo por `pd.NA`

  * Existen múltiples técnicas para **imputar** valores que faltan, ya que los modelos no pueden tratar con ellos directamente

In [1]:
import seaborn as sns

titanic = sns.load_dataset('titanic')

In [2]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Podemos ver el % de valores que faltan en cada columna

In [3]:
titanic.isna().mean() * 100

survived        0.000000
pclass          0.000000
sex             0.000000
age            19.865320
sibsp           0.000000
parch           0.000000
fare            0.000000
embarked        0.224467
class           0.000000
who             0.000000
adult_male      0.000000
deck           77.216611
embark_town     0.224467
alive           0.000000
alone           0.000000
dtype: float64

Podemos eliminar todas las filas que tienen algún NA con pandas:

In [4]:
titanic.shape

(891, 15)

In [5]:
titanic.dropna().shape

(182, 15)

Si una columna tiene un gran porcentaje de valores que faltan, podemos eliminarla y a continuación eliminar todas las filas que tengan algún NA en el resto:

In [6]:
X = titanic.drop(columns=['deck', 'embarked', 'alive', 'survived', 'class', 'who'])
y = titanic['survived']

In [7]:
isna = X.isna().any(axis=1)

In [8]:
X[~isna].shape

(712, 9)

### Imputar valores que faltan

Si tenemos pocos datos o no queremos perder observaciones, en ocasiones es útil completar los valores que faltan de una o más variables. scikit-learn implementa varias estrategias básicas en el módulo `impute`:

  * `impute.SimpleImputer`: puede imputar valores que faltan con la media, mediana, valor más frecuente o una constante
  * `impute.KNNImputer`: imputa usando la media de los $k$ vecinos próximos

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

imputer = make_column_transformer(
    (SimpleImputer(strategy='mean', add_indicator=True), ['age']),
    (SimpleImputer(strategy='most_frequent', add_indicator=True), ['embark_town']),
    remainder='passthrough'
)

X_train_im = imputer.fit_transform(X_train)
X_test_im = imputer.transform(X_test)

In [11]:
X_train_im.shape

(668, 11)

In [12]:
import pandas as pd
pd.DataFrame(X_train_im).isna().mean()

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
dtype: float64

#### Estrategias basadas en modelos

En la versión 0.23 de scikit-learn han añadido `impute.IterativeImputer` (todavía experimental). Funciona de la siguiente manera:

   * Ajusta un modelo donde la salida ($y$) es la variable a imputar y las características con el resto de columnas
   
   * Completa los valores que faltan usando las estimaciones del modelo
   
   * Esto se repite para cada columna donde falten valores

Para transformaciones básicas y/o no incluidas en scikit-learn, también podemos usar pandas. Por ejemplo, para reemplazar los valores que faltan con el más frecuente:

In [13]:
X.loc[X['embark_town'].isna(), 'embark_town'] = X['embark_town'].mode()

Puesto que ahora vamos a imputar valores que faltan usando un modelo, necesitamos que todas las variables sean numéricas:

In [14]:
X_num = pd.get_dummies(X, drop_first=True)

In [15]:
X_num.head()

,pclass,age,sibsp,parch,fare,adult_male,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,True,False,1,0,1
1,1,38.0,1,0,71.2833,False,False,0,0,0
2,3,26.0,0,0,7.9250,False,True,0,0,1
3,1,35.0,1,0,53.1000,False,False,0,0,1
4,3,35.0,0,0,8.0500,True,True,1,0,1


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_num, y, stratify=y, random_state=0)

In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.compose import make_column_transformer

imputer = IterativeImputer()
X_train_ii = imputer.fit_transform(X_train)
X_test_ii = imputer.transform(X_test)

In [18]:
pd.DataFrame({'SimpleImputer': X_train_im[:, 0], 'IterativeImputer': X_train_ii[:, 1]})

,SimpleImputer,IterativeImputer
0,31,31.000000
1,18,18.000000
2,9,9.000000
3,22,22.000000
4,19,19.000000
...,...,...
663,20,20.000000
664,29.7968,25.086903
665,39,39.000000
666,29.7968,22.684674


#### Comparación métodos de imputación

En la documentación se scikit-learn se pueden encontrar dos ejemplos comparando los distintos métodos:

  * [Imputing missing values with variants of IterativeImputer](https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html#sphx-glr-auto-examples-impute-plot-iterative-imputer-variants-comparison-py)
  * [Imputing missing values before building an estimator](https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html#sphx-glr-auto-examples-impute-plot-missing-values-py)
  
Otra comparación visual:

<img src=https://amueller.github.io/ml-workshop-1-of-4/slides/images/med_knn_rf_comparison.png with=500>

### Combinando preprocesos: *Pipelines*

Podemos combinar varios preprocesos para que se realicen sobre distintas columnas (en paralelo) con `ColumnTransformer`

Con la clase `Pipeline`, podemos combinar preprocesos para que se realicen de manera **secuencia**

Los pipelines también nos permiten combinar el preproceso con el ajuste del modelo, por ejemplo para buscar los parámetros óptimos con `GridSearchCV` de forma conjunta 

[Guia de usuario](https://scikit-learn.org/stable/modules/compose.html)

In [19]:
X.head()

,pclass,sex,age,sibsp,parch,fare,adult_male,embark_town,alone
0,3,male,22.0,1,0,7.2500,True,Southampton,False
1,1,female,38.0,1,0,71.2833,False,Cherbourg,False
2,3,female,26.0,0,0,7.9250,False,Southampton,True
3,1,female,35.0,1,0,53.1000,False,Southampton,False
4,3,male,35.0,0,0,8.0500,True,Southampton,True


In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['embark_town', 'sex', 'pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [22]:
X_train_pre = preprocessor.fit_transform(X_train)
X_test_pre = preprocessor.transform(X_test)

In [23]:
X_train_pre.shape

(668, 14)

In [24]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression())])
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8116591928251121

In [25]:
from sklearn import set_config
set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embark_town', 'sex',
                                                   'pclass'])])),
                ('classifier', LogisticRegression())])

### Usando Pipelines en búsquedas de parámetros

Más información y ejemplos: [Pipelines](https://github.com/amueller/ml-workshop-3-of-4/blob/master/notebooks/03%20Pipelines.ipynb)

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'classifier__C': [0.1, 1.0, 10, 100],
}

grid_search = GridSearchCV(clf, param_grid, cv=10)
grid_search.fit(X_train, y_train)
grid_search.score(X_test, y_test)

0.8116591928251121

In [27]:
grid_search.best_params_

{'classifier__C': 10, 'preprocessor__num__imputer__strategy': 'mean'}

### Ejercicios

#### Ejercicio 1

Con los datos del titanic, vamos a intentar predecir la supervivencia ('survived') a partir del resto de variables excepto:

   * `deck`: tiene muchos valores que faltan
    
   * `embarked`: es lo mismo que `embark_town`
    
   * `alive`: es lo mismo que `survived`
   
   * `who`: igual que `sex`
   
   * `class`: igual que `pclass`

Para ello, primero vamos a preparar los datos:

   1. Completar los valores de la variable `embark_town` usando el valor más frecuente
    
   2. Convertir todas las variables a numéricas usando una codificación one-hot


#### Ejercicio 2

Con los datos del ejercicio 1, ahora vamos a ajustar un modelo de regresión logística:
   
   1. Eliminando las filas de los datos donde falta el valor de `age`
   2. Imputando la variable `age` con la media
   3. Imputando la variable `age` usando k vecinos próximos
   4. Imputando la variable `age` usando un modelo de *random forest* (ver parámetro `estimator` de `IterativeImputer`)